# Install Packages and Setup Variables

In [ ]:
!pip install -q llama-index==0.12.21 openai==1.59.8 tiktoken==0.8.0  huggingface-hub==0.30 chromadb==0.6.0 llama-index-vector-stores-chroma==0.4.1 llama-index-llms-gemini==0.4.1 llama-index-embeddings-adapter==0.3.0 llama-index-embeddings-huggingface==0.5.4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 455.6/455.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.1/481.1 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 602.1/602.1 kB 33.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 128.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.8/266.8 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.2 MB/s eta 0

In [ ]:
# set variables
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

os.environ["HF_TOKEN"] = userdata.get('HF_TOKEN2')
HF_TOKEN = userdata.get('HF_TOKEN2')

In [ ]:
# Allows running asyncio in environments with an existing event loop, like Jupyter notebooks.

import nest_asyncio

nest_asyncio.apply()

# Load fine tuned embedding model

In [ ]:
# prompt: download a finetuned model from hugging face, repor vicpada/finetuned_embed_model
from huggingface_hub import snapshot_download

snapshot_download(repo_id="vicpada/finetuned_embed_model_full", repo_type="model", local_dir="./finetuned_embed_model_full")

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.52k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/593k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/55.0 [00:00<?, ?B/s]

'/content/finetuned_embed_model_full'

In [ ]:
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.adapter import AdapterEmbeddingModel

# Load the Base model without fine-tuning
base_embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# Load the Fine-tuned model.
embed_model = AdapterEmbeddingModel(base_embed_model, "finetuned_embed_model_full")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/743 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.core import Settings

Settings.embed_model = embed_model

# Download nodes

In [ ]:
# prompt: download a file from hugging face

snapshot_download(repo_id="vicpada/AzureResources", repo_type="dataset", local_dir="./nodes_full", allow_patterns=['embedded/*'])

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

all_data_embedded.pkl:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

'/content/nodes_full'

In [ ]:
import chromadb
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import VectorStoreIndex



# Load the vector store from the local storage.
chroma_path = './azure-architect'
db = chromadb.PersistentClient(path=chroma_path, settings=chromadb.config.Settings(allow_reset=True))
db.list_collections()
chroma_collection = db.get_collection("azure-architect")
print(chroma_collection.count())
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store)

10350


# Create questions and responses pairs

In [ ]:
import pickle
import os

file_path = 'nodes_full/embedded/all_data_embedded.pkl'

with open(file_path, 'rb') as f:
   all_nodes = pickle.load(f)

len(all_nodes)


195707

In [ ]:
import random

random.shuffle(all_nodes)

eval_nodes = all_nodes[:10000]

In [ ]:
from llama_index.core.evaluation import generate_question_context_pairs
from llama_index.llms.gemini import Gemini
from llama_index.llms.openai import OpenAI

llm = Gemini(model="models/gemini-2.0-flash", temperature=1, max_tokens=512)
#llm = OpenAI(model="gpt-4o-mini", temperature=1, max_tokens=512)
rag_eval_dataset = generate_question_context_pairs(eval_nodes, llm=llm, num_questions_per_chunk=1)

# We can save the dataset as a json file for later use.
rag_eval_dataset.save_json("./rag_eval_dataset2.json")


100%|██████████| 10000/10000 [2:36:13<00:00,  1.07it/s]


In [ ]:
from huggingface_hub import HfApi, HfFolder

api = HfApi(token=HF_TOKEN)

# Specify your Hugging Face repository details
repo_id = "vicpada/AzureRAGEval" # Replace with your username and repository name

# Create the repository if it doesn't exist
api.create_repo(repo_id, repo_type="dataset", exist_ok=True) # Uncomment if creating a new dataset repo

# Upload the zipped file to the repository
api.upload_file(
    path_or_fileobj="rag_eval_dataset2.json",
    path_in_repo="rag_eval_dataset2.json",
    repo_id=repo_id,
    repo_type="dataset",
)

print(f"Successfully uploaded rag_eval_dataset.json to Hugging Face repository: {repo_id}")

rag_eval_dataset2.json:   0%|          | 0.00/27.8M [00:00<?, ?B/s]

Successfully uploaded rag_eval_dataset.json to Hugging Face repository: vicpada/AzureRAGEval
